## Scrape GCN circulars

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
grbs = []
short_flagged = []
for y in range(2004,2020):
    year = str(y)
    main_page = requests.get(f"https://gcn.gsfc.nasa.gov/selected_{year}.html")
    soup = bs(main_page.text, 'html.parser')
    events = soup.find_all('b')
    for event in events:
        if "GRB " in event.text:
            GRB_ID = event.text.split()[1].replace(":","")
            grbs.append(GRB_ID)
            target_url = f"https://gcn.gsfc.nasa.gov/other/{GRB_ID}.gcn3"
            try:
                circulars = requests.get(target_url).text
                if "short burst" in circulars.lower() or "short-burst" in circulars.lower():
                    short_flagged.append(GRB_ID)
            except:
                print("Couldn't get circulars for GRB",GRB_ID)
        
        else:
            pass
                
    print("Finished "+year+".")

In [ ]:
new_ones = pd.read_csv("./products/new.csv",header=None).values.flatten()
for entry in new_ones:
    if "GRB " in entry:
        print(entry.split(":")[0])

In [ ]:
for event in events:
        if "GRB " in event.text:
            GRB_ID = event.text.split()[1].replace(":","")
            grbs.append(GRB_ID)
            target_url = f"https://gcn.gsfc.nasa.gov/other/{GRB_ID}.gcn3"
            try:
                circulars = requests.get(target_url).text
                if "short burst" in circulars.lower() or "short-burst" in circulars.lower():
                    short_flagged.append(GRB_ID)
            except:
                print("Couldn't get circulars for GRB",GRB_ID)
        
        else:
            pass

In [ ]:
not_already = list(set([grb[:6] for grb in short_flagged]) - set([grb[:6] for grb in sGRBs["GRB"]]))
not_already.sort()

In [ ]:
for grb in not_already:
    target_url = f"https://gcn.gsfc.nasa.gov/other/{grb}.gcn3"
    try:
        circulars = requests.get(target_url).text
        print(circulars.split("////////////////////////////////////////////////////////////////////////")[1])
    except:
        print("Couldn't get circulars for GRB",grb)
    print("\n")